In [55]:
%matplotlib inline
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [57]:
# Read dataset
pdata = pd.read_csv("bank.csv")
pdata.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [58]:
pdata.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [59]:
# Drop the columns which are unique for all users
pdata = pdata.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)
pdata.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [60]:
pdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [61]:
# Convert categorical variables
pdata = pd.get_dummies(pdata, columns=['Geography'])
pdata = pd.get_dummies(pdata, columns=['Gender'])
pdata.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [62]:
# Distinguish the features and target variable
X =  pdata.drop(['Exited'], axis=1)
Y = pdata["Exited"]


In [63]:
# Normalize the train and test data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

(8000, 13) (2000, 13) (8000,) (2000,)


In [64]:
# Import libs
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [69]:
# Initialize & build the model
model_classifier = Sequential()
# Add 2 hidden layers
model_classifier.add(Dense(16 ,activation='relu',input_dim = 13))
model_classifier.add(Dense(16,activation='relu'))

# Add output layer
model_classifier.add(Dense(1, activation='sigmoid'))

In [71]:
model_classifier.compile(optimizer='sgd', loss='categorical_crossentropy',  metrics = ['accuracy'])

# model is trained over 100 epochs
model_classifier.fit(X_train,Y_train,epochs=100,validation_data=(X_test,Y_test),batch_size=X_train.shape[0])


Epoch 1/100
1/1 [==============================] - 0s 234ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 2/100
1/1 [==============================] - 0s 40ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 3/100
1/1 [==============================] - 0s 67ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 4/100
1/1 [==============================] - 0s 55ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 5/100
1/1 [==============================] - 0s 68ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 6/100
1/1 [==============================] - 0s 59ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_loss: 2.4140e-08 - val_accuracy: 0.7975
Epoch 7/100
1/1 [==============================] - 0s 81ms/step - loss: 2.4319e-08 - accuracy: 0.7950 - val_los